**LLM:**

In [ ]:
#load reviews
from typing import List
from llama_index.core.readers import StringIterableReader
from llama_index.core.schema import Document
all_reviews.insert(0, "Below are the top 5 reviews of " + dropdown.value + " product")
def load_context_data(context_data: List[str]) -> List[Document]:
    ret: List[Document] = []
    buff: str = ""
    for text in context_data:
        buff += text + "\n\n"
        ret.append(buff)
        buff = ""
    return StringIterableReader().load_data(ret)
tokens = load_context_data(all_reviews)


In [ ]:
from langchain.chains import RetrievalQA
from llama_index.core import VectorStoreIndex, ServiceContext
from llama_index.core import PromptTemplate
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import get_response_synthesizer
from llama_index.llms.huggingface import HuggingFaceInferenceAPI

# load mistralai LLm model
mixtral_llm = HuggingFaceInferenceAPI(
    model_name="mistralai/Mixtral-8x7B-Instruct-v0.1",
    token=huggingface_api_key
)
from llama_index.embeddings.jinaai import JinaEmbedding
jina_embedding_model = JinaEmbedding(
    api_key=Japi_key,
    model="jina-embeddings-v2-base-en",
)

service_context = ServiceContext.from_defaults(
    llm=mixtral_llm, embed_model=jina_embedding_model
)
index = VectorStoreIndex.from_documents(
    documents=tokens, service_context=service_context
)
qa_prompt_tmpl = (
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "answer the query. Please be brief, concise, and complete.\n"
    "Answer: "
)
qa_prompt = PromptTemplate(qa_prompt_tmpl)

# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=2,
)

# configure response synthesizer
response_synthesizer = get_response_synthesizer(
    service_context=service_context,
    text_qa_template=qa_prompt,
    response_mode="compact",
)

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)


In [ ]:
result = query_engine.query("text summary")
print(result.response)